# Model Selection 모듈

- Train/Test 데이터를 분리하지 않고 머신러닝 수행

In [3]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [4]:
iris = load_iris()
dt_clf = DecisionTreeClassifier()
dt_clf.fit(iris.data, iris.target)

DecisionTreeClassifier()

In [5]:
pred = dt_clf.predict(iris.data)
accuracy_score(iris.target, pred)

1.0

- cross_validate() method

In [7]:
from sklearn.model_selection import cross_validate

In [8]:
dtc = DecisionTreeClassifier()
cross_validate(dtc, iris.data, iris.target)

{'fit_time': array([0.00099754, 0.00099659, 0.00099707, 0.00099564, 0.00100088]),
 'score_time': array([0., 0., 0., 0., 0.]),
 'test_score': array([0.96666667, 0.96666667, 0.9       , 1.        , 1.        ])}

In [9]:
dtc = DecisionTreeClassifier()
cross_validate(dtc, iris.data, iris.target, return_train_score=True)

{'fit_time': array([0.00099564, 0.        , 0.        , 0.00099897, 0.        ]),
 'score_time': array([0.        , 0.        , 0.00099516, 0.        , 0.        ]),
 'test_score': array([0.96666667, 0.96666667, 0.9       , 0.96666667, 1.        ]),
 'train_score': array([1., 1., 1., 1., 1.])}

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=2021
)

In [12]:
import pandas as pd
pd.Series(y_test).value_counts()

0    14
1    10
2     6
dtype: int64

- Stratified 분리

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=2021,
    stratify=iris.target
)
pd.Series(y_test).value_counts()

0    10
1    10
2    10
dtype: int64

- cross_val_score() method

In [14]:

from sklearn.model_selection import cross_val_score

In [15]:
# 성능 지표는 정확도(accuracy), 교차검증 세트는 5개
cross_val_score(dtc, iris.data, iris.target, cv=5)

array([0.96666667, 0.96666667, 0.9       , 0.93333333, 1.        ])

In [16]:
import numpy as np 
scores = cross_val_score(dtc, iris.data, iris.target, cv=5)
np.mean(scores)

0.9666666666666668

### GridSearchCV

In [18]:
dtc = DecisionTreeClassifier(random_state=2021)

In [19]:
# parameter 를 dictionary 형태로 설정
params = {
    'max_depth': [2, 3, 4, 5],
    'min_samples_split': [2, 3]
}

In [20]:
from sklearn.model_selection import GridSearchCV
grid_dtc = GridSearchCV(dtc, param_grid=params, cv=3)

In [21]:
grid_dtc.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=2021),
             param_grid={'max_depth': [2, 3, 4, 5],
                         'min_samples_split': [2, 3]})

In [22]:
grid_dtc.cv_results_

{'mean_fit_time': array([0.00066471, 0.00066503, 0.00099564, 0.00099723, 0.00099428,
        0.        , 0.        , 0.00099659]),
 'std_fit_time': array([4.70021655e-04, 4.70246438e-04, 2.70441366e-06, 2.24783192e-07,
        3.95292604e-06, 0.00000000e+00, 0.00000000e+00, 5.15042996e-07]),
 'mean_score_time': array([0.00099691, 0.00066479, 0.        , 0.        , 0.        ,
        0.        , 0.00033307, 0.        ]),
 'std_score_time': array([8.99132768e-07, 4.70077941e-04, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 4.71033179e-04, 0.00000000e+00]),
 'param_max_depth': masked_array(data=[2, 2, 3, 3, 4, 4, 5, 5],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_split': masked_array(data=[2, 3, 2, 3, 2, 3, 2, 3],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'ma

In [23]:
# 최적 파라미터
grid_dtc.best_params_

{'max_depth': 2, 'min_samples_split': 2}

In [24]:
# 최고 정확도
grid_dtc.best_score_

0.9583333333333334

In [25]:
# 최고 정확도를 가지는 최적 파라미터로 학습한 estimator
best_estimator = grid_dtc.best_estimator_
best_estimator.score(X_test, y_test)

0.9